In [1]:
#MAIN ACCESS VARS
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import time
import threading
import json

#Variables that contains the user credentials to access Twitter API
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

In [5]:
#Database connection and handler
import pymongo
import sys
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from datetime import datetime

def connection_try():
    try:
        c = MongoClient(host="localhost", port=27017)
        #return "Connected successfully"
        return c
    except ConnectionFailure, e:
        sys.stderr.write("Could not connect to MongoDB: %s" % e)
        sys.exit(1)
        

def handle_db(client,db_name):
    dbh = client[db_name]
    assert dbh.client == client
    #print "Successfully set up a database handle"
    return dbh

In [6]:
#STREAM LISTENER
#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):

    def __init__(self,alert):
        self.terminate = False
        self.campaignId = alert[0]
        self.campaignName = alert[1]
        self.dbName = 'openMakerInfluencerDeneme'
        #self.c_connect = dbM.connection_try() #try to connect pymongo
        #self.db_listener = dbM.handle_db(self.c_connect,self.dbName) #handle a database
        self.c_connect = connection_try() #try to connect pymongo
        self.db_listener = handle_db(self.c_connect,self.dbName) #handle a database
        super(StdOutListener, self).__init__()

    def on_data(self, data):
        if self.terminate == False:
            tweetInfo = json.loads(data)
            if 'user' in tweetInfo:
                self.db_listener[self.campaignName+str(self.campaignId)].insert_one({'username':tweetInfo['user']['screen_name'],'userid':tweetInfo['user']['id'],'followers_count':tweetInfo['user']['followers_count'],'friends_count':tweetInfo['user']['friends_count']}) #this creates tweets collection, if there is one then write on it
            return True
        else:
            return False

    def on_error(self, status):
        print status
        if self.terminate == True:
            return False

    def stop(self):
        self.terminate = True


class StreamCreator():
    def __init__(self,alert):
        #This handles Twitter authetification and the connection to Twitter Streaming API
        self.l = StdOutListener(alert)
        self.auth = OAuthHandler(consumer_key, consumer_secret)
        self.auth.set_access_token(access_token, access_token_secret)
        self.stream = Stream(self.auth, self.l)
        self.t = threading.Thread(target = self.stream.filter, kwargs = {'track':alert[2],'languages':alert[3]} )
    def start(self):
        self.t.start()
    def checkAlive(self):
        return self.t.isAlive()
    def terminate(self):
        self.l.terminate = True
    def checkTerminated(self):
        return self.l.terminate

In [13]:
arduino_alertList = [1,'ArduinoTheme',['arduino'],['en']]
raspberry_alertList = [1,'RaspberryTheme',['raspberry pi'],['en']]
printer_alertList = [1,'PrinterTheme',['3d print','3d-print','3d printer','3d-printer','3d printing','3d-printing'],['en']]

In [14]:
streamArduino = StreamCreator(arduino_alertList)
streamRaspberry = StreamCreator(raspberry_alertList)
streamPrinter = StreamCreator(printer_alertList)

In [15]:
streamArduino.start()
streamRaspberry.start()
streamPrinter.start()

In [18]:
print streamArduino.checkAlive()
print streamRaspberry.checkAlive()
print streamPrinter.checkAlive()

True
True
True


In [24]:
streamArduino.terminate()
streamRaspberry.terminate()
streamPrinter.terminate()

In [26]:
print streamArduino.checkTerminated()
print streamRaspberry.checkTerminated()
print streamPrinter.checkTerminated()

True
True
True


In [27]:
#DATABASE INFO
dbName = 'openMakerInfluencerDeneme'
c_connect = connection_try()
dB = handle_db(c_connect,dbName)

In [28]:
#Only keep unique users
from bson.objectid import ObjectId

def makeUsersUnique(collectionName):
    userList = list(dB[collectionName].find())
    userIDList = []
    count = 0
    for user in userList:
        if user['userid'] in userIDList:
            dB[collectionName].delete_one({'_id': user['_id']})
        else:
            userIDList.append(user['userid'])

In [29]:
makeUsersUnique('ArduinoTheme1')
makeUsersUnique('RaspberryTheme1')
makeUsersUnique('PrinterTheme1')

In [30]:
#MULTIPLE ACCESS VARS
import pymongo
import sys
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from datetime import datetime
import tweepy
from tweepy import OAuthHandler
import time

Consumer_Key1 = ""
Consumer_Secret1 = ""
Access_Token1 = ""
Access_Token_Secret1 = ""

Consumer_Key2 = "" # API key
Consumer_Secret2 = "" # API secret
Access_Token2 = ""
Access_Token_Secret2 = ""

Consumer_Key3 = ""
Consumer_Secret3 = ""
Access_Token3 = ""
Access_Token_Secret3 = ""

auth1 = OAuthHandler(Consumer_Key1, Consumer_Secret1)
auth1.set_access_token(Access_Token1, Access_Token_Secret1)
api1 = tweepy.API(auth1)

auth2 = OAuthHandler(Consumer_Key2, Consumer_Secret2)
auth2.set_access_token(Access_Token2, Access_Token_Secret2)
api2 = tweepy.API(auth2)

auth3 = OAuthHandler(Consumer_Key3, Consumer_Secret3)
auth3.set_access_token(Access_Token3, Access_Token_Secret3)
api3 = tweepy.API(auth3)

apis = [api1,api2,api3]

In [31]:
#Following Collector
def recognizeRateError(e):
    if e.message[0]['code'] == 88:
        return True #Rate limit error
    else:
        return False
        

def friendsCollector(collectionName):
    userList = list(dB[collectionName].find({"followingList": {"$exists": False}}))
    apiNum = 0
    for user in userList:
        try:
            following_list = list(tweepy.Cursor(apis[apiNum].friends_ids, screen_name=user['username']).items())
            dB[collectionName].update_one({'username':user['username']}, {'$set':{'followingList':following_list}})
        except tweepy.TweepError as e:
            if recognizeRateError(e):
                if apiNum<(len(apis)-1):
                    apiNum = apiNum + 1
                    try:
                        following_list = list(tweepy.Cursor(apis[apiNum].friends_ids, screen_name=user['username']).items())
                        dB[collectionName].update_one({'username':user['username']}, {'$set':{'followingList':following_list}})
                    except:
                        dB[collectionName].update_one({'username':user['username']}, {'$set':{'followingList':[]}})
                        continue
                else:
                    apiNum = 0
                    time.sleep(60 * 15)
                    try:
                        following_list = list(tweepy.Cursor(apis[apiNum].friends_ids, screen_name=user['username']).items())
                        dB[collectionName].update_one({'username':user['username']}, {'$set':{'followingList':following_list}})
                    except:
                        dB[collectionName].update_one({'username':user['username']}, {'$set':{'followingList':[]}})
                        continue
            continue
        print str(user['username']) + " done!"

In [32]:
import threading
arduinoFriendsThread = threading.Thread(target=friendsCollector, args=['ArduinoTheme1'])
raspberryFriendsThread = threading.Thread(target=friendsCollector, args=['RaspberryTheme1'])
printerFriendsThread = threading.Thread(target=friendsCollector, args=['PrinterTheme1'])

In [33]:
arduinoFriendsThread.start()
raspberryFriendsThread.start()
printerFriendsThread.start()

dataforcet done!
Jameleon done!
Linuxfera done!
KemTechie done!
themedssmasher done!
software4iot done!
marskymac done!
ifourdesign done!
ardweetno done!
jotbe done!
1stAngel done!
1paulrichardson done!
imadx10 done!Startup_Sherpas done!

mdwahyu494 done!
milanyud done!


/home/opennote/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
  app.launch_new_instance()


WamdaME done!
Freikaempfer done!
red_lad done!
Noavcu_Reejto done!
SamR1 done!
Goablo_Goayqi done!
Cathy_zhu8 done!
omiez_ done!
alvarolb84 done!
AcquistaFrancis done!
DMioc done!
AnnaHegerC done!
html_css_bot done!
3DPrintMonkey done!
AmazonBay4u done!
bestariwebhost done!
FilipKreisch done!
ErrorTeamWeb done!
giveawayspros done!
Odetteriv done!
AUM4me done!
_phako_ done!
portalarduino done!
nmdis_iti done!
sarah_hasman done!
naderys done!
narfdotpl done!
ruano84 done!
michael_aqt done!
gamecentralfans done!
pollen_robotics done!
NeilForsyth6 done!
fastfixmelb done!
Biappi done!
V2KMGS done!
fabiovenni done!
Coding2Learn done!
EnidKapelsen done!
ctechgames done!
markmulligan done!
dje4816 done!
Karlvaello done!
RobocodeChile done!
sharpless89 done!
MAXMSP_BOT done!
3DPrintMonkey done!
BenBenBeartrax done!
ITCrowdsource done!
Szakulus done!
LattePandaCN done!
jkbhawani done!
InsaneDeals4you done!
Spock444 done!
FragmentedSoul5 done!
GeekyGadgets done!
StackArduino done!
TheMadPro done!

In [36]:
#Rank users with centrality degree
import heapq

dbName = 'openMakerInfluencerDeneme' #DB that data will be read from
c_connect = connection_try()
dB = handle_db(c_connect,dbName)

dbName2 = 'influenceRanks' #DB that data will be written to
c_connect2 = connection_try()
dB2 = handle_db(c_connect2,dbName2)

def centralityRank(collectionName):
    cursor = dB[collectionName].find({"followingList": {"$exists": True}}) #create cursor for loop in mongodb
    rankDic = {}
    #calculate ranking of users
    for user in cursor:
        fList = user['followingList']
        print user['userid']
        for following in fList:
            if str(following) in rankDic:
                rankDic[str(following)] = rankDic[str(following)] + 1
            else:
                rankDic[str(following)] = 1
        #find top 1000 influencer
    k_keys_sorted = heapq.nlargest(1000, rankDic, key=rankDic.get)
    #write to database
    dB2[collectionName].insert_one({'type':'centralityRank', 'rankList':k_keys_sorted})

In [37]:
centralityRank('ArduinoTheme1')

2611409766
4902207238
1458360708
2737
2829568464
326650351
93879997
790580820268417024
786506564228096000
153079720
1190306143
95477106
769629395262771200
1455406592
4876853282
721027647753601024
871830186
3363872993
3306825512
15209124
2427034619
770254611475271681
88356321
2542424130
14811168
14341395
417406645
25309192
606417578
283007984
2434519592
216459818
787419801006247936
208132208
91798669
212176326
4498024934
704823993380032512
819490240327020544
1348347787
833771644976001024
822458499917561856
3952461
288132556
95378589
87973206
432659660
460186362
20620868
300790902
144799984
1461743059
3197146924
808718961986650112
2383737733
778800644396191746
3523170137
833964418727817217
3033181449
801409819316023296
64504412
1153124959
3600579386
805698411513180160
1737042740
3019710229
724076992740515840
812035470762905601
867788725
1707931
11747562
130760661
555832938
766877550090018816
15392736
2698746986
15794494
2974948929
826814475223199745
48572412
822287143590653953
248248876


In [38]:
centralityRank('RaspberryTheme1')

3197146924
96392905
21644907
13568992
707830764587462656
718128753466089476
2551171309
538318253
20901996
54888263
2778216943
819487244977315840
3348679983
707219671293026305
1258116997
3407764475
27602123
2246799980
228378534
140796053
625066761
2562125755
2451966739
43487458
1566664404
821322487447359488
3091398772
822458499917561856
3019503295
4894792367
1696478568
195261336
312430091
26001977
576658081
2279180352
706129621939560448
1663554007
3105443082
785475624408223744
767833341009629184
819767335145746432
1915448922
41209095
206789872
3875339716
18135638
48379730
704831521304150017
1912575216
190427616
3721954701
101699188
2430100928
4276173734
107389994
761995373153955840
14797687
35255262
743435898168057861
118606045
2775347000
3081409336
4105823501
2666790590
2939393247
41602343
807499728489418752
4329396154
160143535
2609555370
1903628527
55480459
828991740052963328
731454132591165440
14197807
2934397885
2697172464
730182223442284544
3236538428
3015128365
737077396331888640

In [39]:
centralityRank('PrinterTheme1')

18600377
833012145088458752
1053160538
701795428484980737
2481531072
152667695
58866887
778538251904614401
2787893402
2733994518
787419801006247936
2209313610
3035472579
777912372262412288
737820681581285376
3708991092
774009074165878786
550556820
2842054474
2354658913
972056354
821318972444409857
62800565
2846486337
371455751
3614866399
151139307
272172398
104520454
3267523640
797835
3760107376
700756530115678208
1032511220
822404856010985473
105931650
741220141602594816
123919107
2238223070
2603474340
2306411718
1176854161
2967986296
3992805111
51247351
1344547279
291912649
2319610428
3925383749
3295029565
3064218203
856340588
4832571802
369093749
730823570440491008
779268927952195588
773531548566126592
3343128257
801769122291806208
1338548911
780654637
717606532662210561
2810050196
809870366613131264
171871800
2798716999
41829156
812244500852641792
3065907929
2396075863
777913627609366528
2498311532
1420571466
827664804068225024
2791864689
1333428170
2920123047
2214261092
393697898


In [40]:
# Filtering users if they really share tweets with content related to theme
dbName = 'influenceRanks' #DB that data will be read from
c_connect = connection_try()
dB = handle_db(c_connect,dbName)

def recognizeRateError(e):
    if e.message[0]['code'] == 88:
        return True #Rate limit error
    else:
        return False
        

def tweetCollector(collectionName,key_word_list,desired_num):
    centralityInfo = dB[collectionName].find_one({'type':'centralityRank'})
    userList = centralityInfo['rankList']
    influencers = []
    apiNum = 0
    count = 0
    for user in userList:
        tweet_list = []
        try:
            tweet_list = apis[apiNum].user_timeline(id=user,count=200)
        except tweepy.TweepError as e:
            if recognizeRateError(e):
                if apiNum<(len(apis)-1):
                    apiNum = apiNum + 1
                    try:
                        tweet_list = apis[apiNum].user_timeline(id=user,count=200)
                    except:
                        continue
                else:
                    apiNum = 0
                    time.sleep(60 * 15)
                    try:
                        tweet_list = apis[apiNum].user_timeline(id=user,count=200)
                    except:
                        continue
            continue
        for tweet in tweet_list:
            found = 0
            tweet_text = tweet.text.lower()
            for key in key_word_list:
                if key in tweet_text:
                    print "###############Related################" + user
                    influencers.append(user)
                    found = 1
                    count = count + 1
                    break
            if found == 1:
                break 
        if count == desired_num:
            break
        print user + " done!"
    dB[collectionName].insert_one({'type':'filteredCentralityRank', 'rankList':influencers})

In [43]:
arduinoTweetsThread = threading.Thread(target=tweetCollector, kwargs = {'collectionName':'ArduinoTheme1','key_word_list':['arduino'],'desired_num':20})

In [44]:
arduinoTweetsThread.start()

###############Related################302666251
302666251 done!
###############Related################266400754
266400754 done!
###############Related################20731304
20731304 done!
###############Related################1118451
1118451 done!
11348282 done!
44196397 done!
###############Related################14607140
14607140 done!
###############Related################1578141
1578141 done!
1344951 done!
###############Related################17877351
17877351 done!
816653 done!
34743251 done!
###############Related################7597362
7597362 done!
20536157 done!
50393960 done!
783214 done!
2916305152 done!
###############Related################767285
767285 done!
###############Related################16734860
16734860 done!
13334762 done!
742143 done!
###############Related################116040561
116040561 done!
###############Related################28542415
28542415 done!
###############Related################138475538
138475538 done!
17471979 done!
19725644 done!
807095

In [45]:
raspberryTweetsThread = threading.Thread(target=tweetCollector, kwargs = {'collectionName':'RaspberryTheme1','key_word_list':['raspberry pi'],'desired_num':20})

In [46]:
raspberryTweetsThread.start()

###############Related################302666251
302666251 done!
816653 done!
11348282 done!
813286 done!
1344951 done!
20536157 done!
44196397 done!
7905122 done!
74286565 done!
14372486 done!
382267114 done!
1118451 done!
95438524 done!
10228272 done!
2803191 done!
783214 done!
759251 done!
266400754 done!
50393960 done!
972651 done!
2890961 done!
428333 done!
###############Related################2784808724
2784808724 done!
21447363 done!
2916305152 done!
16736535 done!
5402612 done!
621583 done!
1513002752 done!
15880163 done!
15473958 done!
243381107 done!
26565946 done!
25073877 done!
16589206 done!
14763734 done!
10671602 done!
4870867247 done!
37599351 done!
26599573 done!
800087125106434048 done!
74594552 done!
###############Related################14607140
14607140 done!
63043 done!
29758446 done!
14375294 done!
19725644 done!
8161232 done!
1541461 done!
###############Related################20731304
20731304 done!
15439395 done!
313835875 done!
108027306 done!
1933042237 done

In [47]:
printerTweetsThread = threading.Thread(target=tweetCollector, kwargs = {'collectionName':'PrinterTheme1','key_word_list':['3d print','3d-print','3d printer','3d-printer','3d printing','3d-printing'],'desired_num':20})

In [48]:
printerTweetsThread.start()

816653 done!
1344951 done!
###############Related################529192106
529192106 done!
50393960 done!
###############Related################1949322637
1949322637 done!
5402612 done!
11348282 done!
###############Related################369093749
369093749 done!
10228272 done!
44196397 done!
###############Related################41580964
41580964 done!
15492359 done!
17093617 done!
###############Related################15536454
15536454 done!
###############Related################22021097
22021097 done!
20536157 done!
###############Related################18870989
18870989 done!
###############Related################587491156
587491156 done!
813286 done!
###############Related################1879821056
1879821056 done!
###############Related################1196309324
1196309324 done!
783214 done!
807095 done!
972651 done!
###############Related################1920498750
1920498750 done!
5988062 done!
###############Related################20009578
20009578 done!
###############Related

In [49]:
#print usernames of influencers
Consumer_Key = ""
Consumer_Secret = ""
Access_Token = ""
Access_Token_Secret = ""

auth = OAuthHandler(Consumer_Key, Consumer_Secret)
auth.set_access_token(Access_Token, Access_Token_Secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

dbName = 'influenceRanks' #DB that data will be read from
c_connect = connection_try()
dB = handle_db(c_connect,dbName)

def findFilteredNames(collectionName):
    centralityInfo = dB[collectionName].find_one({'type':'filteredCentralityRank'})
    infIDs = centralityInfo['rankList']
    influencerList = []
    for key in infIDs:
        user = api.get_user(key)
        influencerList.append(user)
        print user.screen_name

In [50]:
findFilteredNames('ArduinoTheme1')

Raspberry_Pi
arduino
adafruit
make
hackaday
makerfaire
sparkfun
instructables
mbanzi
dangerousproto
freetronics
seeedstudio
MicrochipMakes
makerbot
RealSexyCyborg
element14
planetarduino
MakerShed
arduinoblog
Pi_Borg


In [51]:
findFilteredNames('RaspberryTheme1')

Raspberry_Pi
htpc_guides
hackaday
adafruit
lifehacker
Pi_Borg
pimoroni
RPiSpy
TheMagP1
ThePiHut
TabTimes
androidcentral
CodeClub
RSComponents
xKaliSec
linuxfoundation
element14
StackSocial
makerfaire
ModMyPi


In [52]:
findFilteredNames('PrinterTheme1')

3dprintindustry
3DPrintGirl
3dersorg
3dsystems
shapeways
makerbot
fabbaloo
3DHubs
3DPrintBoard
ColorFabb
My3DPrinting
3dprinting
3d_printers
3dprinting3d
3dprintingcom
Ultimaker
make
imaterialise
3DPrintingFans
3DPrintingIt
